# SW중심대학 디지털 경진대회_SW와 생성AI의 만남 : AI부문
 - 이 AI 경진대회에서는 5초 분량의 오디오 샘플에서 진짜 사람 목소리와 AI가 생성한 가짜 목소리를 정확하게 구분할 수 있는 모델을 개발하는 것이 목표입니다.
 - 이 작업은 보안, 사기 감지 및 오디오 처리 기술 향상 등 다양한 분야에서 매우 중요합니다.

In [ ]:
try:
    import torch
except ImportError:
    try:
        %conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia
    except:
        %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

try:
    import librosa
except:
    try:
        %conda install -c conda-forge librosa
    except:
        %pip install librosa

try:
    import wespeaker
except ImportError:
    %pip install git+https://github.com/wenet-e2e/wespeaker.git

try:
    import huggingface_hub
except ImportError:
    %pip install huggingface_hub

## Imports
모델 학습 및 추론에 사용할 라이브러리들을 불러옵니다.

In [ ]:
import os
import math
import random

import librosa

import torch
import torchaudio

import numpy as np
import pandas as pd

from torch import nn
import torch.nn.functional as F
import torchaudio.transforms as T
import torchaudio.pipelines as pipelines
from torch.utils.data import Dataset, DataLoader

from huggingface_hub import hf_hub_download
import wespeaker

from tqdm.notebook import tqdm

### Check GPU Availability

In [ ]:
!nvidia-smi

In [ ]:
# Set CUDA Device Number 0~7
device_num = 0

if torch.cuda.is_available():
    torch.cuda.set_device(device_num)
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    device_num = -1  # cpu
print(f"INFO: Using device - {device}:{device_num}")

## Config
- 딥러닝 모델을 학습하기 전에 설정해야하는 다양한 매개변수를 정의하는 설정 클래스입니다.
- 클래스를 사용하여 학습에 필요한 설정 값을 미리 지정합니다.

##### 오디오 신호
- 우리가 듣는 소리는 공기의 압력 변화로, 이것을 디지털 신호로 변환한 것이 오디오 신호입니다.
- 이 신호는 시간에 따라 변하는 진폭 값을 가지고 있습니다.

In [ ]:
class Config:
    SEED = 20240719  # 재현성을 위해 랜덤 시드 고정
    NB_NAME = "final"  # ipython 노트북 이름 지정
    DATA_ROOT = os.path.join(".", "data")
    SAMPLE_RATE = 16000
    BATCH_SIZE = 128
    LR = 1e-5

    @staticmethod
    def fix_seed(seed=SEED):
        random.seed(seed)
        os.environ['PYTHONHASHSEED'] = str(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

Config.fix_seed()  # Seed 고정

## Dataset

In [ ]:
from torchvision.datasets import utils
from sklearn.model_selection import train_test_split as split

utils.tqdm = tqdm


class VoiceDataset(Dataset):
    download_url = "https://drive.usercontent.google.com/download?id=1hi1dibkHyFbaxAteLlZJw6r3g9ddd4Lf&export=download&authuser=0&confirm=t&uuid=c40c278b-d74b-4b75-bc79-09e8a3ccffa4&at=APZUnTUvIVFVM9gjGNUCmDb4YZCy%3A1719807236671"
    
    Train = "train"
    Test = "test"
    Unlabeled = "unlabeled_data"

    def download(self, root='./data', filename="download.zip", md5=None):
        filepath = os.path.join(root, filename)
        if not os.path.exists(filepath):
            utils.download_and_extract_archive(self.download_url, root, root, filename, md5)
            print("Extraction completed.")
        else:
            print(f"File already exists in {filepath}")
        return root

    def load_from_source(self, train=True, train_size=1):
        random_state = 1  # fixed random_state

        if os.path.isfile(self.get_dataset_path):
            df = pd.read_csv(self.get_dataset_path)
        else:
            files = os.listdir(self.get_dataset_path.replace(".csv", ""))
            df = pd.DataFrame(dict(
                id=[".".join(*file.split(".")[:-1]) for file in files],
                path=[f"./{self.name}/{file}" for file in files]
            ))

        if 'label' in df.columns:
            df1, df2, _, _ = split(df, df['label'], test_size=train_size, random_state=random_state)
        else:
            df1, df2 = split(df, test_size=train_size, random_state=random_state)

        return df1 if train else df2

    @property
    def get_dataset_path(self):
        return os.path.join(self.download_root, self.name + ".csv")

    @property
    def submission_form_path(self):
        return os.path.join(self.download_root, "sample_submission.csv")

    def __init__(self, root=".", name=None, train=True, train_size=1, transform=None):
        convert_path = lambda path: os.path.join(Config.DATA_ROOT, *path.replace("./", "").split("/"))
        self.download_root = self.download(root)
        super().__init__()
        self.name = name
        
        if isinstance(transform, list) or isinstance(transform, tuple):
            self.transforms = transform
        else:
            self.transforms = [transform] if transform else []

        self.raw_data = self.load_from_source(train, train_size)
        if 'label' in self.raw_data.columns:
            self.label = [(0, 1) if lb == 'real' else (1, 0) for lb in self.raw_data['label'].tolist()]
        else:
            self.label = None
        self.audio = list(map(convert_path, self.raw_data['path'].tolist()))

        print(f"INFO: Dataset Loaded - Name: {self.name}, Size: {len(self.audio)}")

    def transform(self, *item):
        for transform in self.transforms:
            item = transform(*item)
        return item

    @staticmethod
    def load_audio(path):
        waveform, sample_rate = torchaudio.load(path)
        if sample_rate != Config.SAMPLE_RATE:
            waveform = T.Resample(sample_rate, Config.SAMPLE_RATE)(waveform)
        return waveform

    @staticmethod
    def generate_silence(duration=5.0, sample_rate=Config.SAMPLE_RATE):
        num_samples = int(duration * sample_rate)
        silence = torch.zeros((1, num_samples))  # assume mono channel
        return silence

    @staticmethod
    def pad(waveform, pad_size):
        left_pad = 0
        if isinstance(pad_size, list) or isinstance(pad_size, tuple):
            left_pad, pad_size = pad_size
        right_pad = pad_size - waveform.size(1) - left_pad
        return F.pad(waveform, (int(left_pad), int(right_pad)))

    @property
    def augmented(self):
        if not self.label:
            raise NotImplementedError("ERROR: Augmentation is not available for unlabeled dataset.")

        audio_length = 5.0 * Config.SAMPLE_RATE  # fit to test domain audio length
        augmented_list = []

        fakes = [odo for odo, label in zip(self.audio, self.label) if label == (1, 0)]
        reals = [odo for odo, label in zip(self.audio, self.label) if label == (0, 1)]
        fake_random = lambda l=len(fakes)//8: random.shuffle(fakes)[:l]
        real_random = lambda l=len(reals)//8: random.shuffle(reals)[:l]
        rand_length = lambda odo: (random.randrange(0, audio_length - odo.size(1)), audio_length)
        append = augmented_list.append
        
        def reserve(label, data1, data2=None):
            def reserve_inner():
                data1_loaded = self.load_audio(data1)
                data2_loaded = self.load_audio(data2) if data2 else None
                data1_loaded = self.pad(data1_loaded, rand_length(data1_loaded))
                data2_loaded = self.pad(data2_loaded, rand_length(data1_loaded)) if data2 else None
                return self.transforms((data1_loaded + data2_loaded) / 2, label)
            return reserve_inner

        # (1, 0) - [fake 2, real 0]
        for fake1, fake2 in zip(fake_random(), fake_random()):
            append(reserve((1, 0), fake1, fake2))

        # (1, 0) - [fake 1, real 0]
        for fake in fake_random():
            append(reserve((1, 0), fake))

        # (1, 1) - [fake 1, real 1]
        for fake, real in zip(fake_random(len(fakes)//4), real_random(len(reals)//4)):
            append(reserve((1, 0), fake, real))

        # (0, 1) - [fake 0, real 2]
        for real1, real2 in zip(real_random(), real_random()):
            append(reserve((1, 0), real1, real2))

        # (0, 1) - [fake 0, real 1]
        for real in real_random():
            append(reserve((1, 0), real))

        # (0, 0) - [fake 0, real 0]
        for _ in range(min(len(fakes)//4, len(reals)//4)):
            append(lambda: self.transforms(self.generate_silence(duration=audio_length / Config.SAMPLE_RATE), (0, 0)))

        return augmented_list

    def __len__(self):
        return len(self.audio)

    def __getitem__(self, index):
        return self.transform(self.load_audio(self.audio[index]), self.label[index] if self.label else None)

In [ ]:
to_tensor = lambda *args: (*args[:-1], torch.tensor(args[-1]))

train_set = VoiceDataset(root=Config.DATA_ROOT, name=VoiceDataset.Train, train=True, train_size=0.8, transform=to_tensor)
valid_set = VoiceDataset(root=Config.DATA_ROOT, name=VoiceDataset.Train, train=False, train_size=0.8, transform=to_tensor)
unlabeled_set = VoiceDataset(root=Config.DATA_ROOT, name=VoiceDataset.Unlabeled)
test_set = VoiceDataset(root=Config.DATA_ROOT, name=VoiceDataset.Test)

In [ ]:
print("INFO: Query Dataset for checking:", train_set[0][0], train_set[0][0].shape, train_set[0][1])
train_set.raw_data

## DataLoader
    - DataLoader는 구축된 데이터셋에서 배치크기(batch_size)에 맞게 데이터를 추출하고, 필요에 따라 섞거나(shuffle=True) 순서대로 반환(shuffle=False)하는 역할을 합니다.
    - 훈련 데이터(train_loader)는 일반적으로 섞어서 모델이 데이터에 덜 편향되게 학습하도록하며,
      검증 데이터(val_loader)는 모델 성능 평가를 위해 순서대로 사용하고,
      테스트 데이터(test_loader)는 최종적인 추론을 위해 사용합니다.

    이렇게 DataLoader를 사용함으로써, 효율적인 데이터 처리와 모델 학습 및 평가가 가능해집니다.

In [ ]:
def augmented_collate(batch):
    datas, labels = [], []
    for lazy_loader in batch:
        data, label = lazy_loader()
        datas.append(data), labels.append(label)
    return torch.stack(datas, dim=0), torch.tensor([item[1] for item in batch])

In [ ]:
BATCH_SIZE = Config.BATCH_SIZE

#train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
tr_aug_loader = lambda: DataLoader(train_set.augmented, batch_size=BATCH_SIZE, shuffle=True, collate_fn=augmented_collate)
valid_loader = DataLoader(valid_set, batch_size=BATCH_SIZE, shuffle=False)
val_aug_loader = DataLoader(valid_set.augmented, batch_size=BATCH_SIZE, shuffle=False, collate_fn=augmented_collate)
unlabeled_loader = DataLoader(unlabeled_set, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

## Define Model

### 0. Pretrained Embedding Layer

In [ ]:
import torchaudio.compliance.kaldi as kaldi


class ResNet152(nn.Module):
    model_id = "Wespeaker/wespeaker-voxceleb-resnet152-LM"
    model_name = model_id.replace("Wespeaker/wespeaker-", "").replace("-", "_")

    root_dir = hf_hub_download(model_id, filename=model_name+".onnx").replace(model_name+".onnx", "")
    if not os.path.isfile(root_dir+"avg_model.pt"):
        os.rename(hf_hub_download(model_id, filename=model_name+".pt"), root_dir+"avg_model.pt")
    if not os.path.isfile(root_dir+"config.yaml"):
        os.rename(hf_hub_download(model_id, filename=model_name+".yaml"), root_dir+"config.yaml")

    def __init__(self):
        super().__init__()
        self.sample_rate = 16000
        self.embedding_dim = 256
        self.pretrained = wespeaker.load_model_local(self.root_dir).model

    def compute_fbank(self, waveform, num_mel_bins=80, frame_length=25, frame_shift=10, cmn=True):
        features = kaldi.fbank(
            waveform, num_mel_bins=num_mel_bins, frame_length=frame_length, frame_shift=frame_shift,
            sample_frequency=self.sample_rate
        )
        return (features - torch.mean(features, 0)) if cmn else features

    def forward(self, waveform):
        features = self.compute_fbank(waveform.float(), cmn=True).unsqueeze(0)
        embedding = self.model(features.to(self.device))
        return (embedding[-1] if isinstance(embedding, tuple) else embedding)[0]

### 1. Feature Extraction Layer (ResNet-like structure)

In [ ]:
class AttentionLayer(nn.Module):
    def __init__(self, feature_dim):
        super().__init__()
        self.feature_dim = feature_dim
        self.attention_weights = nn.Parameter(torch.randn(feature_dim))

    def forward(self, x):
        attention_scores = F.softmax(self.attention_weights, dim=0)
        weighted_features = x * attention_scores
        return weighted_features

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_features, out_features, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Linear(in_features, out_features)
        self.bn1 = nn.BatchNorm1d(out_features)
        self.relu = nn.LeakyReLU(0.01)
        self.conv2 = nn.Linear(out_features, out_features)
        self.bn2 = nn.BatchNorm1d(out_features)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [ ]:
class FeatureExtractor(nn.Module):
    def __init__(self, embedding_size, hidden_size, latent_size):
        super().__init__()

        # Encoder with skip connections
        self.encoder_block1 = nn.Sequential(
            nn.Linear(embedding_size, hidden_size),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.5),
            nn.BatchNorm1d(hidden_size)
        )
        self.skip1 = nn.Linear(embedding_size, hidden_size)

        self.encoder_block2 = nn.Sequential(
            nn.Linear(hidden_size, hidden_size//2),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.5),
            nn.BatchNorm1d(hidden_size//2)
        )
        self.skip2 = nn.Linear(hidden_size, hidden_size//2)

        self.encoder_block3 = nn.Sequential(
            nn.Linear(hidden_size//2, hidden_size//4),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.5),
            nn.BatchNorm1d(hidden_size//4)
        )
        self.skip3 = nn.Linear(hidden_size//2, hidden_size//4)

        self.encoder_block4 = nn.Sequential(
            nn.Linear(hidden_size//4, hidden_size//8),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.5),
            nn.BatchNorm1d(hidden_size//8)
        )
        self.skip4 = nn.Linear(hidden_size//4, hidden_size//8)

        self.final_encoder = nn.Sequential(
            nn.Linear(hidden_size//8, latent_size),
            nn.LeakyReLU(0.01)
        )

        self.attention = AttentionLayer(latent_size)

    def forward(self, x):
        x1 = self.encoder_block1(x) + self.skip1(x)
        x2 = self.encoder_block2(x1) + self.skip2(x1)
        x3 = self.encoder_block3(x2) + self.skip3(x2)
        x4 = self.encoder_block4(x3) + self.skip4(x3)
        encoded = self.final_encoder(x4)
        attention = self.attention(encoded)
        return attention

In [ ]:
class FeatureExtractor(nn.Module):
    def __init__(self, embedding_size, hidden_size, latent_size):
        super(FeatureExtractor, self).__init__()

        self.initial_layer = nn.Sequential(
            nn.Linear(embedding_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.LeakyReLU(0.01)
        )

        self.layer1 = self._make_layer(hidden_size, hidden_size, 3)
        self.layer2 = self._make_layer(hidden_size, hidden_size//2, 4, stride=2)
        self.layer3 = self._make_layer(hidden_size//2, hidden_size//4, 6, stride=2)
        self.layer4 = self._make_layer(hidden_size//4, hidden_size//8, 3, stride=2)

        self.final_encoder = nn.Sequential(
            nn.Linear(hidden_size//8, latent_size),
            nn.LeakyReLU(0.01)
        )

        self.attention = AttentionLayer(latent_size)

    def _make_layer(self, in_features, out_features, blocks, stride=1):
        downsample = None
        if stride != 1 or in_features != out_features:
            downsample = nn.Sequential(
                nn.Linear(in_features, out_features),
                nn.BatchNorm1d(out_features)
            )

        layers = []
        layers.append(ResidualBlock(in_features, out_features, downsample))
        for _ in range(1, blocks):
            layers.append(ResidualBlock(out_features, out_features))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.initial_layer(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        encoded = self.final_encoder(x)
        attention = self.attention(encoded)
        return attention

### 2. Domain Adaptation Layer

In [ ]:
class DomainAdaptationLayer(nn.Module):
    def __init__(self, feature_dim):
        super().__init__()
        self.feature_map = nn.Sequential(
            nn.Linear(feature_dim, feature_dim),
            nn.ReLU(),
            nn.Linear(feature_dim, feature_dim)
        )

    def forward(self, x):
        return self.feature_map(x)

### 3. Adversarial Domain Classification Layer

In [ ]:
class AdversarialDomainClassifier(nn.Module):
    def __init__(self, feature_dim, hidden_dim=100):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(feature_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)  # output => domain 0(train), 1(test)
        )
    
    def forward(self, x):
        return self.classifier(x)

### 4. Uncertainty-Aware Classification Layer

In [ ]:
class UncertaintyAwareClassifier(nn.Module):
    def __init__(self, feature_dim):
        super().__init__()
        self.classifier = nn.Linear(feature_dim, 2)
        self.uncertainty_threshold = nn.Parameter(torch.tensor([0.5]))

    def forward(self, features):
        logits = self.classifier(features)
        probabilities = F.softmax(logits, dim=1)

        uncertainty = -(probabilities * torch.log(probabilities)).sum(dim=1)
        noise_prob = (uncertainty > self.uncertainty_threshold).float()
        
        return probabilities, noise_prob.unsqueeze(1)

### 6. Total Fake & Real Voice Detection Model

In [ ]:
class VoiceDetector(nn.Module):
    def __init__(self, hidden_size, latent_size):
        super().__init__()
        self.embedding = ResNet152()
        self.embedding_dim = self.embedding.embedding_dim
        #self.domain_adapter = DomainAdaptationLayer(feature_dim=embedding_dim + noise_dim)
        #self.domain_classifier = AdversarialDomainClassifier(feature_dim=embedding_dim + noise_dim)
        self.encoder = FeatureExtractor(
            embedding_size=self.embedding_dim,
            hidden_size=hidden_size,
            latent_size=latent_size
        )
        self.classifier = nn.Linear(latent_size, 2)

    def forward(self, waveform):
        embedding = self.embedding(waveform)
        #domain = self.domain_adapter(embedding)
        #domain = self.domain_classifier(domain)
        features = self.encoder(embedding)
        return F.sigmoid(self.classifier(features))

In [ ]:
# Set model parameters
model_params = dict(
    hidden_size=1024,
    latent_size=128
)
model_params

In [ ]:
# Create model instance
model = VoiceDetector(**model_params)
model.to(device)

In [ ]:
# Freeze Pretrained Embedding Layer
for param in model.embedding.pretrained.parameters():
    param.requires_grad = False

In [ ]:
# BinaryCrossEntropy
criterion = nn.BCELoss()

# Adam optimizer
optimizer = torch.optim.Adam(params=model.parameters(), lr=Config.LR)

## Training & Validation

In [ ]:
from sklearn.metrics import roc_auc_score

def multi_label_auc(y_true, y_scores):
    auc_scores = []
    for idx in range(y_true.shape[1]):
        auc = roc_auc_score(y_true[:, idx], y_scores[:, idx])
        auc_scores.append(auc)
    mean_auc_score = np.mean(auc_scores)
    return mean_auc_score

In [ ]:
def state_manager(num_epochs, log_interval=5):
    cache = [0.0 for _ in range(10)]
    
    def update_state(
            progress, epoch=None, step=None, steps=None, train_loss=None,
            domain_acc=None, domain_loss=None, valid_acc=None, valid_loss=None, cross_acc=None, cross_loss=None
    ):
        new_state = [epoch, step, steps, train_loss, domain_acc, domain_loss, valid_acc, valid_loss, cross_acc, cross_loss]
        for i, state in enumerate(new_state):
            if state:
                cache[i] = state
        epoch, step, steps, train_loss, domain_acc, domain_loss, valid_acc, valid_loss, cross_acc, cross_loss = cache
        progress.update(1)
        print(f"\rEpoch [{epoch}/{num_epochs}], Step [{step}/{steps}], Loss: {train_loss:.6f}, Domain: {domain_acc:.6%} | {domain_loss:.6f}, "
            + f"Valid: {valid_acc:.6%} | {valid_loss:.6f}, Cross Valid: {cross_acc:.6%} | {cross_loss:.6f}", end="   ")
        
        def result():
            print(end="\n" if epoch % log_interval == 0 or epoch == num_epochs else "")
            return cross_acc
        return result
    
    return tqdm(range(1, num_epochs+1), desc="Running Epochs"), update_state

In [ ]:
def infinite_loader(loader):
    while True:
        for data in loader:
            yield data

In [ ]:
num_epochs = 50
epochs, update_state = state_manager(num_epochs, log_interval=5)

performance = 0
loaders = (tr_aug_loader, unlabeled_loader, val_aug_loader, valid_loader)
lens = list(map(len, loaders))

with (tqdm(total=lens[0], desc="Training") as tr_pgrs, tqdm(total=lens[2], desc="Validation") as val_pgrs, tqdm(total=lens[3], desc="Cross Validation") as crv_pgrs):
    for epoch in epochs:
        [p.reset(total=l) for p, l in zip((tr_pgrs, val_pgrs, crv_pgrs), lens)]  # progressbar reset

        # Train
        model.train()
        for step, train_inputs in enumerate(loaders[0]):
            optimizer.zero_grad()

            features, labels = (data.float().to(device) for data in train_inputs)
            outputs = model(features)

            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()

            update_state(tr_pgrs, epoch, step, lens[0], loss.item())

        # Cross Domain Validation
        model.eval()
        with torch.no_grad():
            for loader, loader_len, progress, cross in zip(loaders[-2:], lens[-2:], (val_pgrs, crv_pgrs), (False, True)):
                val_loss, val_labels, val_outputs = 0, [], []
                
                for inputs in loader:
                    features, labels = (data.float().to(device) for data in inputs)
                    predicted = model(features)

                    val_loss += criterion(predicted, labels).item() / loader_len
                    val_labels.append(labels.cpu().numpy())
                    val_outputs.append(predicted.cpu().numpy())
                    val_acc = multi_label_auc(np.concatenate(val_labels, axis=0), np.concatenate(val_outputs, axis=0))

                    if cross:
                        result = update_state(progress, cross_acc=val_acc, cross_loss=val_loss)
                    else:
                        update_state(progress, valid_acc=val_acc, valid_loss=val_loss)

        performance = result()

### Model Save

In [ ]:
if not os.path.isdir(os.path.join(".", "models")):
    os.mkdir(os.path.join(".", "models"))

# Model Save
save_path = os.path.join(".", "models", f"{Config.NB_NAME}_acc_{performance*100:.6f}.pt")
torch.save(model.state_dict(), save_path)
print(f"Model saved to {save_path}")

## Inference
테스트 데이터셋에 대한 추론은 다음 순서로 진행됩니다.

1. 모델 및 디바이스 설정
    - 모델을 주어진 device(GPU 또는 CPU)로 이동시키고, 평가모드로 전환합니다.
2. 예측 수행
    - 예측 결과를 저장한 빈 리스트를 초기화하고 test_loader에서 배치별로 데이터를 불러와 예측을 수행합니다.
    - 각 배치에 대해 스펙트로그램 데이터를 device로 이동시킵니다.
    - 모델 예측 확률(probs)을 계산합니다.
    - 예측 확률을 predictions리스트에 추가합니다.

In [ ]:
predicted_labels = []

model.to(device)
model.eval()
with torch.no_grad():
    for features in tqdm(test_loader):
        (probs, _), _ = model(*map(lambda x: x.to(device), features))
        probs = probs.cpu().detach().numpy()
        predicted_labels += probs.tolist()

### Submission
추론 결과를 제출 양식에 덮어 씌워 CSV 파일로 생성하는 과정은 다음과 같습니다.

1. 제출 양식 로드
    - pd.read_csv('./sample_submission.csv')를 사용하여 제출을 위한 샘플 형식 파일을 로드합니다.
    - 이 파일은 일반적으로 각 테스트 샘플에 대한 ID와 예측해야 하는 필드가 포함된 템플릿 형태를 가지고 있습니다.
2. 예측 결과 할당
    - submit.iloc[:,1:] = preds 추론함수(inference)에서 반환된 예측결과(preds)를 샘플 제출 파일에 2번째 열부터 할당합니다.
3. 제출 파일 저장
    - 수정된 제출 파일을 baseline_submit 이란 이름의 CSV 파일로 저장합니다.
    - index=False는 파일 저장시 추가적인 index가 발생하지 않도록 설정하여, 제작한 제출 파일과 동일한 형태의 파일을 저장합니다.

In [ ]:
submit = pd.read_csv(test_set.submission_form_path)
submit.iloc[:, 1:] = predicted_labels
submit.head()

In [ ]:
submission_dir = "submissions"
if not os.path.isdir(submission_dir):
    os.mkdir(submission_dir)

submit_file_path = os.path.join(".", submission_dir, f"{Config.NB_NAME}_acc_{performance*100:.6f}_submit.csv")
submit.to_csv(submit_file_path, index=False)
print("File saved to", submit_file_path)